In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
from fastai import *
from fastai.vision import *
from pathlib import Path
import os
import pandas as pd


In [ ]:
print(os.listdir("../input"))


In [ ]:
train_dir = "../input/train/train/"
test_dir = "../input/test/test/"


In [ ]:
print(os.listdir(train_dir)[:5])
print(os.listdir(test_dir)[:5])


In [ ]:
train_csv = pd.read_csv("../input/train.csv")
sample_submission = pd.read_csv("../input/sample_submission.csv")

display(train_csv.head())
display(sample_submission.head())


In [ ]:
tfms = get_transforms()
data = ImageDataBunch.from_df(
    df = train_csv,
    path = train_dir,
    test = "../../test",
    valid_pct = 0.2,
    bs = 16,
    size = 32,
    ds_tfms = tfms,
    num_workers = 0
).normalize(imagenet_stats)
print(data.classes)
data.show_batch(rows=2)


In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy, model_dir="/tmp/models")


In [ ]:
learn.fit_one_cycle(1)


In [ ]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)


In [ ]:
class_score, y = learn.get_preds(DatasetType.Test)
class_score = np.argmax(class_score, axis=1)


In [ ]:
submission  = pd.DataFrame({
    "id": os.listdir(test_dir),
    "has_cactus": class_score
})
submission.to_csv("submission.csv", index=False)
submission[:5]
